In [1]:
!pip install mysql-connector-python


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import mysql.connector

In [3]:
conn=mysql.connector.connect(
    host="localhost",
    user="root",
    password="Harddik1234",
    database="supply_chain"
)

In [4]:
conn.is_connected()

True

In [5]:
# we create a function as connection to the database and return the connection object
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="Harddik1234",
        database="supply_chain"
    )

In [6]:
connect_to_db().is_connected()

True

In [7]:
db=connect_to_db()
cursor=db.cursor(dictionary=True)

In [8]:
cursor.execute("SELECT count(*) FROM suppliers")
row=cursor.fetchone()
print(row)

{'count(*)': 50}


In [9]:
queries = {
"Total Suppliers": "SELECT COUNT(*) AS count FROM suppliers",

"Total Products": "SELECT COUNT(*) AS count FROM products",

"Total Categories Dealing": "SELECT COUNT(DISTINCT category) AS count FROM products",

"Total Sale Value (Last 3 Months)": """
SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_sale
FROM stock_entries se
JOIN products p ON se.product_id = p.product_id
WHERE se.change_type = 'Sale'
AND se.entry_date >= (
SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
""",

"Total Restock Value (Last 3 Months)": """
SELECT ROUND(SUM(se.change_quantity * p.price), 2) AS total_restock
FROM stock_entries se
JOIN products p ON se.product_id = p.product_id
WHERE se.change_type = 'Restock'
AND se.entry_date >= (
SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
""",

"Below Reorder & No Pending Reorders": """
SELECT COUNT(*) AS below_reorder
FROM products p
WHERE p.stock_quantity < p.reorder_level
AND p.product_id NOT IN (
SELECT DISTINCT product_id FROM reorders WHERE status = 'Pending')
"""
}

In [10]:
result={}
for label, query in queries.items():
    cursor.execute(query)
    row= cursor.fetchone()
    result[label]=list(row.values())[0]

In [11]:
result

{'Total Suppliers': 50,
 'Total Products': 200,
 'Total Categories Dealing': 5,
 'Total Sale Value (Last 3 Months)': 1529482.99,
 'Total Restock Value (Last 3 Months)': 6691663.1,
 'Below Reorder & No Pending Reorders': 14}

In [12]:
def get_basic_info(cursor):
    queries = {
        "Total Suppliers": "SELECT COUNT(*) AS count FROM suppliers",

        "Total Products": "SELECT COUNT(*) AS count FROM products",

        "Total Categories Dealing": "SELECT COUNT(DISTINCT category) AS count FROM products",

        "Total Sale Value (Last 3 Months)": """
        SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_sale
        FROM stock_entries se
        JOIN products p ON se.product_id = p.product_id
        WHERE se.change_type = 'Sale'
        AND se.entry_date >= (
        SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
        """,

        "Total Restock Value (Last 3 Months)": """
        SELECT ROUND(SUM(se.change_quantity * p.price), 2) AS total_restock
        FROM stock_entries se
        JOIN products p ON se.product_id = p.product_id
        WHERE se.change_type = 'Restock'
        AND se.entry_date >= (
        SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
        """,

        "Below Reorder & No Pending Reorders": """
        SELECT COUNT(*) AS below_reorder
        FROM products p
        WHERE p.stock_quantity < p.reorder_level
        AND p.product_id NOT IN (
        SELECT DISTINCT product_id FROM reorders WHERE status = 'Pending')
        """
        }
    
    result={}
    for label, query in queries.items():
        cursor.execute(query)
        row= cursor.fetchone()
        result[label]=list(row.values())[0]

    return result

In [ ]:
def get_additional_tables(cursor):
    queries={
        "Suppliers contact details": "SELECT supplier_name, contact_name,email,phone FROM suppliers",
        "Product with Supplier and Stock": """ Select p.product_name, s.supplier_name, p.stock_quantity,p.reorder_level FROM products p 
        JOIN suppliers s ON p.supplier_id = s.supplier_id
        order by p.stock_quantity asc
        """
    }

    tables={}
    for label, query in queries.items():
        cursor.execute(query)
        rows= cursor.fetchall()
        tables[label]=pd.DataFrame(rows)

In [18]:
tables

NameError: name 'tables' is not defined